# Python for Public Transport

by Alex Raichev  
for Kiwi PyCon 2019   
in Wellington, New Zealand   


## My background

- Do like math, algorithms, and Python
- Work at MRCagney in Auckland, a walking, cycling, public transport, and planning consultancy
- Get to use Python there to analyze and visualize civic data


## Intro

- Three main kinds of public transport data: 
    * **Scheduling**: when and where the vehicles should be
    * **Realtime**: when and where the vehicles actually are
    * **Ticketing**: who is riding the vehicels
- Focus here on **scheduling data** 


## General Transit Feed Specification (GTFS)

- Most popular **open standard** for encoding scheduling data
- Developed in 2005 by Portland's TriMet transit agency and Google
- Some uses
    * Routing, ala Google Maps
    * Measuring performance
    * Answering whimsical questions


## Running Example

- A recent GTFS feed for Wellington downloadable from [TransitFeeds](http://transitfeeds.com/p/metlink/22)
- [Visit TransitFeeds and explore the feed to understand the spec]


## Challenge

Use the Wellington GTFS feed and Python to answer the following questions:

- What is the most/least frequent route?
- What is the shortest/longest distance route?
- What is the slowest/fastest route?


## Approach

- Pythonistas, how to handle all those CSVs?
- ``csv`` module?
- Database?
- Suggestions?

- [GTFSTK](https://github.com/mrcagney/gtfstk), an open source A Python 3.6+ tool kit for analyzing GTFS data
- It's the best library for this sort of thing
- Disclaimer: i wrote it

In [29]:
from pathlib import Path

import gtfstk as gt
import pandas as pd


DATA_DIR = Path("../data")
%ls {DATA_DIR}

# Set study date
DATE = "20190408"  # Monday


wellington_gtfs_20190331.zip


In [5]:
# Read GTFS feed

feed = gt.read_gtfs(DATA_DIR/"wellington_gtfs_20190331.zip", dist_units="km")
feed.describe()

,indicator,value
0,agencies,"[Valley Flyer, Mana Coach Services Ltd, Metlin..."
1,timezone,Pacific/Auckland
2,start_date,20190331
3,end_date,20191231
4,num_routes,98
5,num_trips,8621
6,num_stops,2884
7,num_shapes,319
8,sample_date,20190404
9,num_routes_active_on_sample_date,86


In [6]:
# Validate

feed.validate()

/home/araichev/.virtualenvs/python_for_public_transport/lib/python3.6/site-packages/gtfstk/validators.py:1422: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  g = pd.concat([g, feed.calendar_dates])


,type,message,table,rows
1,warning,"Repeated pair (trip_id, departure_time)",stop_times,"[172594, 172663, 172732, 172801, 172870, 172939]"
0,warning,Stop has no stop times,stops,"[2870, 2871, 2872, 2873, 2874, 2875, 2876, 287..."
2,warning,Unrecognized column from_trip_id,transfers,[]
3,warning,Unrecognized column to_trip_id,transfers,[]


## Pandas play

In [55]:
feed.trips.T



,0,1,2,3,4,5,6,7,8,9,...,8611,8612,8613,8614,8615,8616,8617,8618,8619,8620
route_id,320,320,320,320,320,320,320,320,320,320,...,6,6,6,6,140,140,830,830,1210,1210
service_id,229,143,127,137,102,159_2,217_1,241,218,139,...,Su+Hol,Rail Sa,Su+Hol,Su+Hol,MTuWThF-XSch,MTuWThF-XSch,MTuWThF-XSch,MTuWThF-XSch,MTuWThF-XSch,MTuWThF-XSch
trip_id,32x__0__103__TZM__229__229,32x__0__111__TZM__143__143,32x__0__123__TZM__127__127,32x__0__119__TZM__137__137,32x__0__117__TZM__102__102,32x__0__115__TZM__159__2__159__2,32x__0__107__TZM__217__1__217__1,32x__0__101__TZM__241__241,32x__0__121__TZM__218__218,32x__0__109__TZM__139__139,...,JVL__1__9323__RAIL__Su+Hol,JVL__1__9367__RAIL__Rail_Sa,JVL__0__9358__RAIL__Su+Hol,JVL__0__9328__RAIL__Su+Hol,14__1__198__NBM__MTuWThF-XSch,14__0__201__NBM__MTuWThF-XSch,83__0__161__NBM__MTuWThF-XSch,83__0__159__NBM__MTuWThF-XSch,121__0__145__TZM__MTuWThF-XSch,121__1__142__TZM__MTuWThF-XSch
trip_headsign,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
direction_id,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,1,0,0,0,0,1
block_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shape_id,[@356.0.16077778@]2,[@356.0.16077778@]2,[@356.0.16077778@]2,[@356.0.16077778@]2,[@356.0.16077778@]2,[@356.0.16077778@]2,[@356.0.16077778@]2,[@356.0.16077778@]2,[@356.0.16077778@]2,[@356.0.16077778@]2,...,[@6.0.20608239@]JVL:801#24315#24322,[@6.0.20608239@]JVL:801#24315#24322,[@6.0.20608239@]JVL:803#24333#24340,[@6.0.20608239@]JVL:803#24333#24340,[@356.0.30312352@]5,[@356.0.30312352@]2,[@356.0.30313467@]83:382#12651#12715,[@356.0.30313467@]83:382#12651#12715,[@356.0.42934812@]1,[@356.0.42934812@]2
wheelchair_accessible,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bikes_allowed,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [56]:
# Only show route, trip, and shape columns: slice columns, AKA filter
feed.trips.filter(["route_id", "trip_id", "shape_id"]).T


,0,1,2,3,4,5,6,7,8,9,...,8611,8612,8613,8614,8615,8616,8617,8618,8619,8620
route_id,320,320,320,320,320,320,320,320,320,320,...,6,6,6,6,140,140,830,830,1210,1210
trip_id,32x__0__103__TZM__229__229,32x__0__111__TZM__143__143,32x__0__123__TZM__127__127,32x__0__119__TZM__137__137,32x__0__117__TZM__102__102,32x__0__115__TZM__159__2__159__2,32x__0__107__TZM__217__1__217__1,32x__0__101__TZM__241__241,32x__0__121__TZM__218__218,32x__0__109__TZM__139__139,...,JVL__1__9323__RAIL__Su+Hol,JVL__1__9367__RAIL__Rail_Sa,JVL__0__9358__RAIL__Su+Hol,JVL__0__9328__RAIL__Su+Hol,14__1__198__NBM__MTuWThF-XSch,14__0__201__NBM__MTuWThF-XSch,83__0__161__NBM__MTuWThF-XSch,83__0__159__NBM__MTuWThF-XSch,121__0__145__TZM__MTuWThF-XSch,121__1__142__TZM__MTuWThF-XSch
shape_id,[@356.0.16077778@]2,[@356.0.16077778@]2,[@356.0.16077778@]2,[@356.0.16077778@]2,[@356.0.16077778@]2,[@356.0.16077778@]2,[@356.0.16077778@]2,[@356.0.16077778@]2,[@356.0.16077778@]2,[@356.0.16077778@]2,...,[@6.0.20608239@]JVL:801#24315#24322,[@6.0.20608239@]JVL:801#24315#24322,[@6.0.20608239@]JVL:803#24333#24340,[@6.0.20608239@]JVL:803#24333#24340,[@356.0.30312352@]5,[@356.0.30312352@]2,[@356.0.30313467@]83:382#12651#12715,[@356.0.30313467@]83:382#12651#12715,[@356.0.42934812@]1,[@356.0.42934812@]2


In [59]:
# Find all trips of a specific route: slice rows
feed.trips.loc[lambda x: x.route_id == "6"].T


,8399,8400,8401,8402,8403,8404,8405,8406,8407,8408,...,8605,8606,8607,8608,8609,8610,8611,8612,8613,8614
route_id,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
service_id,Rail Sa,Rail MTuWThF-XHol,Rail MTuWThF-XHol,Su+Hol,Rail MTuWThF-XHol,Rail Sa,Rail Sa,Su+Hol,Su+Hol,Su+Hol,...,Rail MTuWThF-XHol,Rail Sa,Rail MTuWThF-XHol,Rail MTuWThF-XHol,Rail MTuWThF-XHol,Su+Hol,Su+Hol,Rail Sa,Su+Hol,Su+Hol
trip_id,JVL__1__9331__RAIL__Rail_Sa,JVL__1__9225__RAIL__Rail_MTuWThF-XHol,JVL__0__9250__RAIL__Rail_MTuWThF-XHol,JVL__0__9326__RAIL__Su+Hol,JVL__1__9263__RAIL__Rail_MTuWThF-XHol,JVL__0__9336__RAIL__Rail_Sa,JVL__0__9392__RAIL__Rail_Sa,JVL__1__9367__RAIL__Su+Hol,JVL__1__9355__RAIL__Su+Hol,JVL__1__9333__RAIL__Su+Hol,...,JVL__1__9221__RAIL__Rail_MTuWThF-XHol,JVL__0__9362__RAIL__Rail_Sa,JVL__0__9222__RAIL__Rail_MTuWThF-XHol,JVL__0__9232__RAIL__Rail_MTuWThF-XHol,JVL__0__9256__RAIL__Rail_MTuWThF-XHol,JVL__1__9381__RAIL__Su+Hol,JVL__1__9323__RAIL__Su+Hol,JVL__1__9367__RAIL__Rail_Sa,JVL__0__9358__RAIL__Su+Hol,JVL__0__9328__RAIL__Su+Hol
trip_headsign,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
direction_id,1,1,0,0,1,0,0,1,1,1,...,1,0,0,0,0,1,1,1,0,0
block_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shape_id,[@6.0.20608239@]JVL:801#24315#24322,[@6.0.20608239@]JVL:801#24315#24322,[@6.0.20608239@]JVL:803#24333#24340,[@6.0.20608239@]JVL:803#24333#24340,[@6.0.20608239@]JVL:801#24315#24322,[@6.0.20608239@]JVL:803#24333#24340,[@6.0.20608239@]JVL:803#24333#24340,[@6.0.20608239@]JVL:801#24315#24322,[@6.0.20608239@]JVL:801#24315#24322,[@6.0.20608239@]JVL:801#24315#24322,...,[@6.0.20608239@]JVL:801#24315#24322,[@6.0.20608239@]JVL:803#24333#24340,[@6.0.20608239@]JVL:803#24333#24340,[@6.0.20608239@]JVL:803#24333#24340,[@6.0.20608239@]JVL:803#24333#24340,[@6.0.20608239@]JVL:801#24315#24322,[@6.0.20608239@]JVL:801#24315#24322,[@6.0.20608239@]JVL:801#24315#24322,[@6.0.20608239@]JVL:803#24333#24340,[@6.0.20608239@]JVL:803#24333#24340
wheelchair_accessible,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bikes_allowed,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [60]:
# Group all trips by route: group
feed.trips.groupby("route_id").get_group("320").T


,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
route_id,320,320,320,320,320,320,320,320,320,320,...,320,320,320,320,320,320,320,320,320,320
service_id,229,143,127,137,102,159_2,217_1,241,218,139,...,148,131,160_1,204_2,201_1,218,216_1,101_2,137,140
trip_id,32x__0__103__TZM__229__229,32x__0__111__TZM__143__143,32x__0__123__TZM__127__127,32x__0__119__TZM__137__137,32x__0__117__TZM__102__102,32x__0__115__TZM__159__2__159__2,32x__0__107__TZM__217__1__217__1,32x__0__101__TZM__241__241,32x__0__121__TZM__218__218,32x__0__109__TZM__139__139,...,32x__1__112__TZM__148__148,32x__1__104__TZM__131__131,32x__1__120__TZM__160__1__160__1,32x__1__106__TZM__204__2__204__2,32x__1__102__TZM__201__1__201__1,32x__1__118__TZM__218__218,32x__1__116__TZM__216__1__216__1,32x__1__114__TZM__101__2__101__2,32x__1__108__TZM__137__137,32x__1__110__TZM__140__140
trip_headsign,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
direction_id,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
block_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shape_id,[@356.0.16077778@]2,[@356.0.16077778@]2,[@356.0.16077778@]2,[@356.0.16077778@]2,[@356.0.16077778@]2,[@356.0.16077778@]2,[@356.0.16077778@]2,[@356.0.16077778@]2,[@356.0.16077778@]2,[@356.0.16077778@]2,...,[@356.0.16077778@]1,[@356.0.16077778@]1,[@356.0.16077778@]1,[@356.0.16077778@]1,[@356.0.16077778@]1,[@356.0.16077778@]1,[@356.0.16077778@]1,[@356.0.16077778@]1,[@356.0.16077778@]1,[@356.0.16077778@]1
wheelchair_accessible,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bikes_allowed,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [61]:
# Compute number of trips per route: split, apply, combine
def my_agg(group):
    d = {}
    d["num_trips"] = group.shape[0]
    return pd.Series(d)

# Use method chaining
(
    feed.trips
    # Split
    .groupby("route_id")
    # Apply and combine
    .apply(my_agg)
    .reset_index()
    # Join in route short name from feed.routes
    .merge(feed.routes.filter(["route_id", "route_short_name", "route_long_name", "route_type"]))
    # Sort
    .sort_values("num_trips", ascending=False)
    .T
)


,87,0,62,26,81,8,1,21,12,37,...,53,94,31,29,30,33,60,52,47,64
route_id,9,10,30,20,70,1200,1100,180,1300,2200,...,270,9918,2030,2010,2020,2060,2910,2640,2510,3000
num_trips,498,424,416,399,308,268,265,262,249,243,...,4,4,3,3,3,3,2,2,2,2
route_short_name,CCL,1,3,2,7,120,110,18,130,220,...,27,N8,203,201,202,206,291,264,251,300
route_long_name,Cable Car (Kelburn - Wellington),Johnsonville West/Churton Park/Grenada Village...,Lyall Bay/Rongotai - Kilbirnie - Newtown - Wel...,Karori - Wellington - Hataitai - Seatoun,Kingston - Brooklyn - Wellington,Stokes Valley - Taita - Epuni - Lower Hutt,Emerald Hill - Upper Hutt - Lower Hutt - Petone,Miramar - Miramar Shops,Naenae - Waterloo - Lower Hutt - Petone,Titahi Bay - Porirua - Ascot Park,...,Vogeltown - Wellington,After Midnight (Lower Hutt - Petone - Wellington),Masterton - Masterton North - Masterton,Masterton - Masterton West - Masterton,Masterton - Masterton South - Masterton,Masterton - Masterton East - Masterton,Waikanae - Levin,Paraparaumu East - Paraparaumu - Kapiti Health...,Paekakariki - Paraparaumu - Kapiti Health Centre,Whenua Tapu Cemetery - Porirua - Titahi Bay
route_type,5,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3


## Let's put GTFSTK to work


In [18]:
# Fill in shape_dist_traveled column of stop_times for later distance calculations.
trip_stats = feed.compute_trip_stats()

feed = feed.append_dist_to_stop_times(trip_stats)
feed.stop_times.T

,0,1,2,3,4,5,6,7,8,9,...,299467,299468,299469,299470,299471,299472,299473,299474,299475,299476
trip_id,110__0__101__TZM__521__521,110__0__101__TZM__521__521,110__0__101__TZM__521__521,110__0__101__TZM__521__521,110__0__101__TZM__521__521,110__0__101__TZM__521__521,110__0__101__TZM__521__521,110__0__101__TZM__521__521,110__0__101__TZM__521__521,110__0__101__TZM__521__521,...,WRL__1__1693__RAIL__Special_Xmas,WRL__1__1693__RAIL__Special_Xmas,WRL__1__1693__RAIL__Special_Xmas,WRL__1__1693__RAIL__Special_Xmas,WRL__1__1693__RAIL__Special_Xmas,WRL__1__1693__RAIL__Special_Xmas,WRL__1__1693__RAIL__Special_Xmas,WRL__1__1693__RAIL__Special_Xmas,WRL__1__1693__RAIL__Special_Xmas,WRL__1__1693__RAIL__Special_Xmas
arrival_time,23:07:00,23:07:28,23:08:16,23:08:39,23:09:31,23:10:56,23:11:36,23:12:45,23:14:01,23:14:38,...,06:27:00,06:38:00,06:48:00,06:53:00,07:02:00,07:23:00,07:32:00,07:50:00,07:58:00,08:10:00
departure_time,23:07:00,23:07:28,23:08:16,23:08:39,23:09:31,23:10:56,23:11:36,23:12:45,23:14:01,23:14:38,...,06:27:00,06:38:00,06:48:00,06:53:00,07:02:00,07:23:00,07:32:00,07:50:00,07:58:00,08:10:00
stop_id,9000,9002,9003,9004,9005,9007,9008,9009,9110,9111,...,SOLW,CART,MATA,WOOD,FEAT,MAYM,UPPE,WATE1,PETO1,WELL
stop_sequence,0,1,2,3,4,5,6,7,8,9,...,2,3,4,5,6,7,8,9,10,11
stop_headsign,Emerald Hill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WELL - Non stop,NaN
pickup_type,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
drop_off_type,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
shape_dist_traveled,0,0.202915,0.541467,0.711481,1.06714,1.638,1.91583,2.38393,2.89787,3.1613,...,2.85028,14.349,21.3971,25.8837,33.8883,52.2671,58.5921,75.5015,80.4869,90.9659
timepoint,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
# Trip stats
trip_stats.T

,2518,2519,2520,2614,2521,2522,2523,2615,2675,2524,...,8531,8532,8533,8534,8545,8546,8547,8548,8549,8550
trip_id,1__0__101__TZM__101__2__101__2,1__0__103__TZM__104__104,1__0__105__TZM__109__1__109__1,1__0__401__TZM__101__3__101__3,1__0__107__TZM__115__115,1__0__109__TZM__123__123,1__0__111__TZM__124__124,1__0__403__TZM__106__1__106__1,1__0__701__TZM__101__4__101__4,1__0__113__TZM__203__1__203__1,...,N66__0__101__TZM__605__1__605__1,N66__0__101__TZM__605__4__605__4,N66__0__103__TZM__605__1__605__1,N66__0__103__TZM__605__4__605__4,N88__0__101__NBM__1__1__1__1,N88__0__101__NBM__1__2__1__2,N88__0__103__NBM__2__1__2__1,N88__0__103__NBM__2__2__2__2,N88__0__105__NBM__2__1__2__1,N88__0__105__NBM__2__2__2__2
route_id,10,10,10,10,10,10,10,10,10,10,...,9966,9966,9966,9966,9988,9988,9988,9988,9988,9988
route_short_name,1,1,1,1,1,1,1,1,1,1,...,N66,N66,N66,N66,N88,N88,N88,N88,N88,N88
route_type,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
direction_id,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
shape_id,[@356.0.16077561@]7,[@356.0.16077561@]12,[@356.0.16077561@]7,[@356.0.16077561@]7,[@356.0.16077561@]12,[@356.0.16077561@]7,[@356.0.16077561@]12,[@356.0.16077561@]7,[@356.0.16077561@]7,[@356.0.16077561@]10,...,[@6.0.22109575@]N66:18#729#814,[@6.0.22109575@]N66:18#729#814,[@6.0.22109575@]N66:18#729#814,[@6.0.22109575@]N66:18#729#814,[@356.0.30313526@]N88:20#819#882,[@356.0.30313526@]N88:20#819#882,[@356.0.30313526@]N88:20#819#882,[@356.0.30313526@]N88:20#819#882,[@356.0.30313526@]N88:20#819#882,[@356.0.30313526@]N88:20#819#882
num_stops,53,63,53,53,63,53,63,53,53,58,...,83,83,83,83,66,66,66,66,66,66
start_time,05:40:00,06:00:00,06:20:00,06:35:00,06:40:00,06:55:00,07:05:00,07:05:00,07:05:00,07:15:00,...,01:30:00,01:30:00,03:00:00,03:00:00,00:30:00,00:30:00,02:00:00,02:00:00,03:30:00,03:30:00
end_time,06:40:00,06:59:00,07:20:00,07:39:00,07:39:00,08:00:00,08:10:00,08:09:00,08:09:00,08:20:00,...,02:30:00,02:30:00,04:00:00,04:00:00,01:30:00,01:30:00,03:00:00,03:00:00,04:30:00,04:30:00
start_stop_id,7158,7158,7158,7158,7158,7158,7158,7158,7158,7158,...,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000


In [40]:
# Route stats
route_stats = feed.compute_route_stats(trip_stats, dates=[DATE])
route_stats.T

,0,1,2,3,4,5,6,7,8,9,...,75,76,77,78,79,80,81,82,83,84
route_id,10,1100,1110,1120,1130,1140,1150,120,1200,121,...,6,600,601,70,8,810,830,840,850,9
route_short_name,1,110,111,112,113,114,115,12,120,12e,...,JVL,60,60e,7,WHF,81,83,84,85x,CCL
route_type,3,3,3,3,3,3,3,3,3,3,...,2,3,3,3,4,3,3,3,3,5
num_trips,192,107,29,39,8,31,18,66,112,11,...,90,68,11,139,32,29,58,13,5,182
num_trip_starts,192,107,29,39,8,31,18,66,112,11,...,90,68,11,139,32,29,58,13,5,182
num_trip_ends,188,107,29,39,8,31,18,65,112,11,...,90,68,11,138,32,29,57,13,5,182
is_loop,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
is_bidirectional,1,1,0,1,1,1,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
start_time,05:40:00,05:38:00,06:22:00,06:10:00,06:55:00,06:20:00,06:30:00,05:52:00,05:55:00,06:40:00,...,05:32:00,06:40:00,06:25:00,06:00:00,06:20:00,05:55:00,06:05:00,06:45:00,06:55:00,07:00:00
end_time,24:45:00,23:09:00,19:02:00,19:13:00,18:28:00,18:50:00,18:28:00,24:00:00,22:51:00,19:02:00,...,23:53:00,21:54:00,18:51:00,24:23:00,19:55:00,19:30:00,24:24:00,18:30:00,18:14:00,22:05:00


In [62]:
# Join in route long name
route_stats = route_stats.merge(feed.routes.filter(["route_id", "route_long_name"]))

cols = [
    "route_id",
    "route_short_name", 
    "route_long_name", 
    "route_type", 
    "start_time",
    "end_time",
    "num_trips",
    "max_headway",
    "mean_trip_distance",
    "mean_trip_duration",
    "service_speed",
]


In [63]:
# The most/least frequent routes are
(
    route_stats
    .filter(cols)
    .sort_values("max_headway")
    .T
)


,84,26,78,60,74,73,72,0,67,66,...,31,69,32,62,47,24,80,58,77,51
route_id,9,20,70,30,580,570,560,10,360,350,...,2030,4,2040,310,260,191,810,291,601,2640
route_short_name,CCL,2,7,3,58,57,56,1,36,35,...,203,WRL,204,31x,26,19e,81,29e,60e,264
route_long_name,Cable Car (Kelburn - Wellington),Karori - Wellington - Hataitai - Seatoun,Kingston - Brooklyn - Wellington,Lyall Bay/Rongotai - Kilbirnie - Newtown - Wel...,Newlands - Wellington,Woodridge - Wellington,Johnsonville - Paparangi - Wellington,Johnsonville West/Churton Park/Grenada Village...,Lyall Bay - Kilbirnie - Hataitai - Wellington,Hataitai - Wellington,...,Masterton - Masterton North - Masterton,Wairarapa Line (Masterton - Wellington),Greytown - Woodside,Miramar North - Wellington (Express),Khandallah - Ngaio - Brandon Street,Johnsonville - Churton Park - Johnsonville (We...,Eastbourne - Petone - Wellington,Newtown - Southgate - Owhiro Bay - Brooklyn (W...,Porirua - Tawa - Johnsonville - Wellington,Paraparaumu East - Paraparaumu - Kapiti Health...
route_type,5,3,3,3,3,3,3,3,3,3,...,3,2,3,3,3,3,3,3,3,3
start_time,07:00:00,06:00:00,06:00:00,06:00:00,06:34:00,06:32:00,06:22:00,05:40:00,05:47:00,07:20:00,...,09:55:00,05:46:00,05:56:00,06:26:00,06:37:00,05:50:00,05:55:00,06:45:00,06:25:00,10:25:00
end_time,22:05:00,24:22:00,24:23:00,24:23:00,18:32:00,18:26:00,18:22:00,24:45:00,18:36:00,18:22:00,...,14:50:00,20:03:00,19:55:00,19:12:00,18:55:00,19:18:00,19:30:00,18:38:00,18:51:00,13:50:00
num_trips,182,182,139,180,14,14,13,192,24,10,...,3,10,10,23,31,16,29,16,11,2
max_headway,10,15,16,17,19,19,19,19,20,20,...,190,308,308,424,435,438,450,487,496,NaN
mean_trip_distance,0.636442,18.1043,6.78669,9.18586,14.02,13.3994,14.7166,22.0115,7.21254,6.13191,...,11.7909,90.9667,11.3971,10.6905,9.00953,18.4278,27.7377,13.7094,24.7543,4.77594
mean_trip_duration,0.0833333,0.959524,0.461391,0.645185,0.778571,0.728571,0.787179,1.14071,0.558333,0.506667,...,0.5,1.72667,0.26,0.59058,0.41129,0.801042,0.963218,0.810417,1.04242,0.333333


In [52]:
# The shortest/longest routes are
(
    route_stats
    .filter(cols)
    .sort_values("mean_trip_distance")
    .T
)



,84,21,18,53,51,33,34,42,66,2,...,83,80,10,1,70,81,28,25,57,69
route_id,9,180,170,280,2640,2060,210,232,350,1110,...,850,810,1210,1100,5,830,2000,2,2900,4
route_short_name,CCL,18,17,28,264,206,21,23z,35,111,...,85x,81,121,110,HVL,83,200,KPL,290,WRL
route_long_name,Cable Car (Kelburn - Wellington),Miramar - Miramar Shops,Kowhai Park - Brooklyn,Beacon Hill - Strathmore Park Shops,Paraparaumu East - Paraparaumu - Kapiti Health...,Masterton - Masterton East - Masterton,Karori (Wrights Hill) - Kelburn - Courtenay Place,Wellington Zoo - Wellington,Hataitai - Wellington,Upper Hutt - Totara Park - Upper Hutt,...,Eastbourne - Wellington (Express),Eastbourne - Petone - Wellington,Stokes Valley Heights - Naenae - Lower Hutt - ...,Emerald Hill - Upper Hutt - Lower Hutt - Petone,Hutt Valley Line (Upper Hutt - Wellington),Eastbourne - Lower Hutt - Petone - Wellington,Masterton - Greytown - Featherston - Martinbor...,Kapiti Line (Waikanae - Wellington),Waikanae - Otaki - Waikanae,Wairarapa Line (Masterton - Wellington)
route_type,5,3,3,3,3,3,3,3,3,3,...,3,3,3,3,2,3,3,2,3,2
start_time,07:00:00,06:13:00,06:20:00,06:57:00,10:25:00,10:25:00,06:25:00,09:05:00,07:20:00,06:22:00,...,06:55:00,05:55:00,05:45:00,05:38:00,04:30:00,06:05:00,05:55:00,05:00:00,06:26:00,05:46:00
end_time,22:05:00,23:22:00,23:58:00,18:40:00,13:50:00,15:15:00,23:54:00,18:13:00,18:22:00,19:02:00,...,18:14:00,19:30:00,19:37:00,23:09:00,23:50:00,24:24:00,19:50:00,24:14:00,19:18:00,20:03:00
num_trips,182,104,60,15,2,3,127,14,10,29,...,5,29,35,107,111,58,20,117,9,10
max_headway,10,127,35,30,NaN,170,30,60,20,49,...,105,450,70,35,30,60,170,30,147,308
mean_trip_distance,0.636442,2.03534,3.16134,3.69959,4.77594,4.90849,5.55176,5.8886,6.13191,6.37243,...,27.1515,27.7377,29.2399,30.2342,30.4569,32.1331,40.5565,47.879,50.0839,90.9667
mean_trip_duration,0.0833333,0.0894231,0.151389,0.166667,0.333333,0.25,0.283465,0.519048,0.506667,0.223563,...,1.01667,0.963218,0.934762,1.08785,0.685736,1.13908,0.849167,0.865812,1.09074,1.72667


In [53]:
# The slowest/fastest routes are
(
    route_stats
    .filter(cols)
    .sort_values("service_speed")
    .T
)


,84,42,66,20,41,67,52,60,51,63,...,50,17,54,32,70,57,59,28,69,25
route_id,9,232,350,171,231,360,270,30,2640,320,...,2620,1600,2800,2040,5,2900,3,2000,4,2
route_short_name,CCL,23z,35,17e,23e,36,27,3,264,32x,...,262,160,280,204,HVL,290,MEL,200,WRL,KPL
route_long_name,Cable Car (Kelburn - Wellington),Wellington Zoo - Wellington,Hataitai - Wellington,Kowhai Park - Brooklyn - Wellington,Houghton Bay - Newtown - Wellington,Lyall Bay - Kilbirnie - Hataitai - Wellington,Vogeltown - Wellington,Lyall Bay/Rongotai - Kilbirnie - Newtown - Wel...,Paraparaumu East - Paraparaumu - Kapiti Health...,Houghton Bay - Island Bay - Berhampore - Welli...,...,Paraparaumu Beach - Paraparaumu (via Mazengarb...,Wainuiomata North - Waterloo - Lower Hutt,Waikanae - Waikanae Beach - Waikanae,Greytown - Woodside,Hutt Valley Line (Upper Hutt - Wellington),Waikanae - Otaki - Waikanae,Melling Line (Melling - Wellington),Masterton - Greytown - Featherston - Martinbor...,Wairarapa Line (Masterton - Wellington),Kapiti Line (Waikanae - Wellington)
route_type,5,3,3,3,3,3,3,3,3,3,...,3,3,3,3,2,3,2,3,2,2
start_time,07:00:00,09:05:00,07:20:00,07:10:00,06:40:00,05:47:00,07:20:00,06:00:00,10:25:00,06:40:00,...,06:08:00,05:48:00,05:23:00,05:56:00,04:30:00,06:26:00,06:14:00,05:55:00,05:46:00,05:00:00
end_time,22:05:00,18:13:00,18:22:00,18:44:00,19:33:00,18:36:00,18:47:00,24:23:00,13:50:00,19:09:00,...,21:34:00,23:54:00,21:50:00,19:55:00,23:50:00,19:18:00,18:55:00,19:50:00,20:03:00,24:14:00
num_trips,182,14,10,11,19,24,4,180,2,24,...,52,78,30,10,111,9,46,20,10,117
max_headway,10,60,20,30,54,20,60,17,NaN,30,...,60,37,73,308,30,147,70,170,308,30
mean_trip_distance,0.636442,5.8886,6.13191,7.41276,9.02705,7.21254,7.00111,9.18586,4.77594,10.1157,...,9.45167,18.9499,17.2168,11.3971,30.4569,50.0839,14.1404,40.5565,90.9667,47.879
mean_trip_duration,0.0833333,0.519048,0.506667,0.598485,0.720175,0.558333,0.541667,0.645185,0.333333,0.702083,...,0.3,0.599359,0.463333,0.26,0.685736,1.09074,0.299275,0.849167,1.72667,0.865812


In [64]:
# Map our extreme routes
feed.map_routes(route_ids=["9", "601", "4", "2"])

# Some open-source Python tools for public transport

- [GTFSTK](https://github.com/mrcagney/gtfstk), library for analyzing GTFS feeds
- [transitfeeds-api](https://github.com/fitnr/transitfeeds-api), Python wrapper for the transitfeeds.com API 
- [django-multi-gtfs](https://github.com/tulsawebdevs/django-multi-gtfs), Django app to import and export GTFS feeds
- [partridge](https://github.com/remix/partridge), fast, forgiving Python GTFS reader built on Pandas DataFrames
- [peartree](https://github.com/kuanb/peartree), library for converting transit data into a directed graph for network analysis 
- And more at the Github repository [awesome-transit](https://github.com/CUTR-at-USF/awesome-transit)